# Cache the mp_terms associated with each marker symbol

In [1]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir('../')

print(os.getcwd())

/mnt/e/Research/TSUMUGI/notebooks/notebools-web
/mnt/e/Research/TSUMUGI


In [5]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import json

In [3]:
path_data = Path("data", "statistical_filtered.csv")

In [4]:
data = pd.read_csv(path_data)

/tmp/ipykernel_1549466/2449997823.py:1: DtypeWarning: Columns (25,60) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path_data)


## Connect marker_symbol to mp_term_name

In [5]:
data_select = data[['marker_symbol', 'mp_term_name']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,mp_term_name
0,Uap1,abnormal lens morphology
1,6430548M08Rik,enlarged kidney
2,Stac,abnormal locomotor behavior
3,Slc24a4,enlarged thymus
4,Uhrf2,female infertility
...,...,...
54048,Lrrk1,abnormal joint morphology
54051,Rtl5,abnormal kidney morphology
54052,Adcy3,increased total body fat amount
54055,Alg3,"preweaning lethality, complete penetrance"


In [6]:
data_dict = defaultdict(list)
for index, row in data_select.iterrows():
    data_dict[row['marker_symbol']].append(row['mp_term_name'])

for key in data_dict.keys():
    data_dict[key].sort()

print(data_dict["Dpf2"])

['abnormal craniofacial morphology', 'abnormal embryo size', 'abnormal heart morphology', 'abnormal kidney morphology', 'abnormal limb morphology', 'abnormal seminal vesicle morphology', 'abnormal tail morphology', 'edema', 'pallor', 'preweaning lethality, complete penetrance']


In [ ]:
pickle.dump(data_dict, open("data/marker_mp.pkl", "wb"))

## Connect mp_term_name to IMPC URL

In [6]:
data_select = data[['mp_term_id', 'mp_term_name']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,mp_term_id,mp_term_name
0,MP:0000186,decreased circulating HDL cholesterol level
1,MP:0011100,"preweaning lethality, complete penetrance"
3,MP:0013279,increased fasting circulating glucose level
4,MP:0000194,increased circulating calcium level
5,MP:0005011,increased eosinophil cell number
...,...,...
44618,MP:0001562,abnormal circulating calcium level
46141,MP:0003655,absent pancreas
46159,MP:0005498,hyporesponsive to tactile stimuli
46562,MP:0008206,increased B-2 B cell number


In [7]:
data_dict_url = dict()
for index, row in data_select.iterrows():
    mp_tern_id = row['mp_term_id']
    impc_url = f"https://www.mousephenotype.org/data/phenotypes/{mp_tern_id}"
    mp_term_name = row['mp_term_name']
    data_dict_url[mp_term_name] = impc_url

print(data_dict_url["small lymph nodes"])

https://www.mousephenotype.org/data/phenotypes/MP:0002217


In [ ]:
with open('data/mp_term_name_id_url.tsv', 'w') as f:
    for term, url in data_dict_url.items():
        f.write(f"{term}\t{url}\n")

In [ ]:
%%bash

head data/mp_term_name_id_url.tsv

## Connect marker_symbol to accession_id

In [8]:
data_select = data[['marker_symbol', 'marker_accession_id']].drop_duplicates()
# data_select = data[['marker_symbol', 'marker_accession_id', 'mp_term_name', 'mp_term_id']].drop_duplicates()
data_select

,marker_symbol,marker_accession_id
0,Rhd,MGI:1202882
1,Dpf2,MGI:109529
2,Gna13,MGI:95768
3,Dynlrb2,MGI:1922715
4,Slc16a2,MGI:1203732
...,...,...
46753,Amigo3,MGI:2444854
46806,Sec1,MGI:1928893
46833,Slc23a3,MGI:104516
46894,Agtr2,MGI:87966


In [9]:
data_dict = dict()
for index, row in data_select.iterrows():
    data_dict[row['marker_symbol']] = row['marker_accession_id']

In [10]:
data_dict["Ncam1"]

'MGI:97281'

In [ ]:
json.dump(data_dict, open("data/marker_symbol_accession_id.json", "w"))
Path("data/marker_symbol_accession_id.tsv").write_text("\n".join([f"{k}\t{v}" for k, v in data_dict.items()]))